# Introduction to SciPy
Tutorial at EuroSciPy 2019, Bilbao

## Getting started

We first import a number of packages.

In [ ]:
import numpy as np
np.__version__

In [ ]:
import scipy
scipy.__version__

In [ ]:
from scipy import fftpack, optimize, signal, stats

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

## Rocking motion of a TGV

Our data are stored in a compressed file `TGV_data.csv.bz2`. Each row of the uncompressed file contains entries separated by commas and the first row contains labels explaining the content of the respective column.

NumPy provides rather universal tools to import data from files into NumPy arrays. We will use `genfromtxt` which allows to deal with `bz2` compressed data files and also handles the labels in the first row of the data.

In [ ]:
np.info(np.genfromtxt)

In [ ]:
data = np.genfromtxt('TGV_data.csv.bz2', delimiter=',', names=True)

There are five columns identified by names:

In [ ]:
data.dtype.names

In [ ]:
time = data['Time_s']
omega_x = data['Gyroscope_x_rads']
omega_y = data['Gyroscope_y_rads']

Let us first get an idea of the data.

In [ ]:
plt.plot(time, omega_x)
plt.plot(time, omega_y)

### Statistical analysis

We use the data for $\omega_x$ to demonstrate some statistical analysis. Let us first take a look at a histogram of the data.

In [ ]:
n, bins = np.histogram(omega_x, bins=100, density=True)

Because we have set `density=True`, the data can be considered as a normalized probability distribution.

In [ ]:
np.sum(n)*(bins[1]-bins[0])

The array `bins` contains the edges of the bins. To plot the histogram, we need their centers.

In [ ]:
bincenters = 0.5*(bins[1:]+bins[:-1])

In [ ]:
plt.plot(bincenters, n)

With SciPy we can easily determine some statistical characteristics from the original data:

In [ ]:
description = stats.describe(omega_x, ddof=False)
description

We can compare our histogram with a Gaussian for the mean and variance just obtained.

In [ ]:
loc = description.mean
scale = np.sqrt(description.variance)
loc, scale

In [ ]:
x = np.linspace(-0.1, 0.1, 200)
plt.plot(x, stats.norm.pdf(x, loc, scale))

A maximum likelihood fit with a Gaussian will yield the same result.

In [ ]:
stats.norm.fit(omega_x)

How can we fit the histogram to a Gaussian? Use nonlinear least-squares curve fitting.

In [ ]:
def gaussian(x, loc, scale):
    return stats.norm.pdf(x, loc, scale)

In [ ]:
popt, pcov = optimize.curve_fit(gaussian, bincenters, n)
popt

In [ ]:
plt.plot(x, gaussian(x, *popt))

How likely is it that our data follow a normal distribution?

In [ ]:
stats.normaltest(omega_x)

Normally distributed data

In [ ]:
normdata = stats.norm.rvs(1, 0.2, 5000)
plt.plot(normdata)

In [ ]:
n, bins = np.histogram(normdata, bins=100, density=True)
bincenters = 0.5*(bins[1:]+bins[:-1])
plt.plot(bincenters, n)

In [ ]:
x = np.linspace(0, 1.6, 200)
plt.plot(x, stats.norm.pdf(x, *stats.norm.fit(normdata)))

In [ ]:
popt, pcov = optimize.curve_fit(gaussian, bincenters, n)
popt

In [ ]:
plt.plot(x, gaussian(x, *popt))

In [ ]:
stats.normaltest(normdata)

Distribution with skewness

In [ ]:
for a in range(-4, 5, 2):
    x = np.linspace(stats.skewnorm.ppf(0.001, a),
                    stats.skewnorm.ppf(0.999, a), 100)
    plt.plot(x, stats.skewnorm.pdf(x, a))

### Spectral properties

Before analyzing the data any further, let us take a look at the time elapsed between subsequent measurements.

In [ ]:
time_intervals = time[1:]-time[:-1]

In [ ]:
_ = plt.hist(time_intervals, bins=100)

We will ignore these small differences and assume the data to be equally spaced in time with the mean time difference between subsequent data points.

In [ ]:
delta_t = np.mean(time_intervals)
delta_t

We will do the spectral analysis with the data for $\omega_y$.

In [ ]:
plt.plot(*signal.periodogram(omega_y, 1/delta_t))

In [ ]:
z = fftpack.rfft(omega_y)

In [ ]:
plt.plot(z[::2])

In [ ]:
plt.plot(z[1::2])

In [ ]:
freqs = fftpack.rfftfreq(omega_y.shape[0], delta_t)

In [ ]:
plt.plot(freqs[::2], z[::2])

In [ ]:
plt.plot(freqs[1::2], z[1::2])

We are interested in the signal around 1.4 Hz. Filter out frequencies beyond 2 Hz.

In [ ]:
filter_coeffs = signal.firwin(301, 3, pass_zero=True, fs=1/delta_t)

In [ ]:
freqs, response = signal.freqz(filter_coeffs, fs=1/delta_t)

In [ ]:
plt.plot(freqs, 20*np.log10(np.abs(response)))

In [ ]:
plt.polar(np.angle(response), np.abs(response))

In [ ]:
plt.plot(time, omega_y)

In [ ]:
omega_y_filtered = signal.convolve(omega_y, filter_coeffs, mode='valid')

In [ ]:
omega_y.shape, omega_y_filtered.shape

In [ ]:
plt.plot(time[150:-150], omega_y[150:-150])

In [ ]:
plt.plot(time[150:-150], omega_y_filtered)

Filter out anything but the range from 53-70 Hz

In [ ]:
filter_coeffs = signal.firwin(301, [53, 70], pass_zero='bandpass', fs=1/delta_t)

In [ ]:
freqs, response = signal.freqz(filter_coeffs, fs=1/delta_t)

In [ ]:
plt.plot(freqs, 20*np.log10(abs(response)))

In [ ]:
omega_y_filtered = signal.convolve(omega_y, filter_coeffs, mode='valid')

In [ ]:
plt.plot(time[150:-150], omega_y[150:-150])
plt.plot(time[150:-150], omega_y_filtered)

In [ ]:
plt.plot(fftpack.rfftfreq(omega_y_filtered.shape[0], delta_t)[::2],
         fftpack.rfft(omega_y_filtered)[::2])

In [ ]:
freq, sp_time, Sxx = signal.spectrogram(omega_y, fs=1/delta_t, nperseg=512)
plt.pcolormesh(freq, sp_time, Sxx)

In [ ]:
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(freq[:, np.newaxis], time, Sxx, cmap=cm.viridis)